In [6]:
from goatools import obo_parser
from zipfile import ZipFile

import os
import gzip
import graco
import shutil
import requests
import numpy as np
import pandas as pd
import networkx as nx
import Bio.UniProt.GOA as GOA

In [3]:
pd.set_option("display.max_columns", 50)

DATA_DIRECTORY = "/home/clusterduck123/Desktop/git/supplements/data"
CPP_DIRECTORY = "/home/clusterduck123/Desktop/git/graco/graco/cpp"

In [4]:
RAW_DATA_DIRECTORY = f"{DATA_DIRECTORY}/raw_data"
PPI_DIRECTORY = f"{DATA_DIRECTORY}/PPI"
ANNOTATIONS_DIRECTORY = f"{DATA_DIRECTORY}/annotations"
MATRIX_DIRECTORY = f"{DATA_DIRECTORY}/matrix"
CLUSTERS_DIRECTORY = f"{DATA_DIRECTORY}/clusters"

if not os.path.exists(DATA_DIRECTORY):
    os.makedirs(DATA_DIRECTORY)
        
if not os.path.exists(RAW_DATA_DIRECTORY):
    os.makedirs(RAW_DATA_DIRECTORY)     
    
if not os.path.exists(PPI_DIRECTORY):
    os.makedirs(PPI_DIRECTORY)
    
if not os.path.exists(ANNOTATIONS_DIRECTORY):
    os.makedirs(ANNOTATIONS_DIRECTORY)
    
if not os.path.exists(MATRIX_DIRECTORY):
    os.makedirs(MATRIX_DIRECTORY)
    
if not os.path.exists(CLUSTERS_DIRECTORY):
    os.makedirs(CLUSTERS_DIRECTORY)

# Downloads

### BioGRID

In [4]:
if not os.path.exists(RAW_DATA_DIRECTORY):
     os.makedirs(directory)

BioGRID_FILENAME = "BIOGRID-ORGANISM-3.5.177.tab2.zip"
BioGRID_URL = "https://downloads.thebiogrid.org/Download/BioGRID/Release-Archive/BIOGRID-3.5.177"
BioGRID_FILEPATH = f"{RAW_DATA_DIRECTORY}/{BioGRID_FILENAME}" 

In [5]:
# Download
r = requests.get(f"{BioGRID_URL}/{BioGRID_FILENAME}", allow_redirects=True)
with open(BioGRID_FILEPATH, 'wb') as f:
    f.write(r.content)

In [6]:
# Unzip Saccharomyces cerevisiae file
with ZipFile(BioGRID_FILEPATH, 'r') as z:
    BioGRID_sc_FILENAME, = [name for name in z.namelist() if 'cerevisiae' in name.lower()]
    z.extract(BioGRID_sc_FILENAME, RAW_DATA_DIRECTORY)

### SGD annotations

In [9]:
SGD_FILENAME = "sgd.gaf.gz"
SGD_URL = "http://current.geneontology.org/annotations"
SGD_FILEPATH = f"{RAW_DATA_DIRECTORY}/{SGD_FILENAME}"

In [10]:
# Download
r = requests.get(f"{SGD_URL}/{SGD_FILENAME}", allow_redirects=True)
with open(SGD_FILEPATH, 'wb') as f:
    f.write(r.content)

### GO

In [11]:
GO_FILENAME = "go-basic.obo"
GO_URL = "http://purl.obolibrary.org/obo/go"
GO_FILEPATH = f"{RAW_DATA_DIRECTORY}/{GO_FILENAME}"

In [12]:
# Download
r = requests.get(f"{GO_URL}/{GO_FILENAME}", allow_redirects=True)
with open(GO_FILEPATH, 'wb') as f:
    f.write(r.content)

# BioGRID PPI (S. cerevisiae)

## Define interactor universes

In [7]:
# load BioGRID file as dataframe
organism_FILENAME = "BIOGRID-ORGANISM-Saccharomyces_cerevisiae_S288c-3.5.177.tab2.txt"
organism_FILEPATH = f"{RAW_DATA_DIRECTORY}/{organism_FILENAME}"

# remove entrez ids and scores because of mixed datatypes 
BioGRID_df = pd.read_csv(organism_FILEPATH, delimiter='\t',
                           usecols=[index for index in range(24) if index not in {1,2,18}])

In [8]:
# BioGRID universe is the collection of every gene known to BioGRID
universe = set(BioGRID_df['BioGRID ID Interactor A']) | \
           set(BioGRID_df['BioGRID ID Interactor B'])

systematic_universe = set(BioGRID_df['Systematic Name Interactor A']) | \
                      set(BioGRID_df['Systematic Name Interactor B'])
systematic_universe.remove('-')

In [9]:
# Filter for (reliable) physical interactions
EXPERIMENTAL_SYSTEM = {'Two-hybrid', 
                       'Affinity Capture-Luminescence',
                       'Affinity Capture-MS', 
                       'Affinity Capture-RNA', 
                       'Affinity Capture-Western'}
EXPERIMENTAL_SYSTEM_TYPE = {'physical'} # redundant because of experimental evidence filtering

physical_interaction_df = BioGRID_df[BioGRID_df['Experimental System'].isin(EXPERIMENTAL_SYSTEM)]
physical_interaction_df = physical_interaction_df[
    physical_interaction_df['Experimental System Type'].isin(EXPERIMENTAL_SYSTEM_TYPE)] # just in case...


physical_universe = set(physical_interaction_df['BioGRID ID Interactor A']) | \
                    set(physical_interaction_df['BioGRID ID Interactor B'])

systematic_physical_universe = set(physical_interaction_df['Systematic Name Interactor A']) | \
                                       set(physical_interaction_df['Systematic Name Interactor B'])
systematic_physical_universe.remove('-')

## Define PPI

In [10]:
# PPI in dataframe
PPI_df = physical_interaction_df[
    physical_interaction_df["Systematic Name Interactor A"].str.startswith('Y') & \
    physical_interaction_df["Systematic Name Interactor B"].str.startswith('Y')]

PPI_universe = set(PPI_df['Systematic Name Interactor A']) | \
               set(PPI_df['Systematic Name Interactor B'])

In [11]:
# Reduce PPI to simple network
PPI_nx = nx.from_pandas_edgelist(PPI_df,'Systematic Name Interactor A', 'Systematic Name Interactor B')
PPI_nx.remove_edges_from(nx.selfloop_edges(PPI_nx))

### Summary

In [12]:
print("BioGRID universe sizes:")
print("=========================")
print(f"    -         -    : {len(universe)}")
print(f"systematic    -    : {len(systematic_universe)}")
print(f"    -      physical: {len(physical_universe)}")
print(f"systematic physical: {len(systematic_physical_universe)}")
print("-------------------------")
print(f"PPI population size: {PPI_nx.number_of_nodes()}")

BioGRID universe sizes:
    -         -    : 7172
systematic    -    : 6535
    -      physical: 6551
systematic physical: 6121
-------------------------
PPI population size: 5726


### Save

In [13]:
nx.write_edgelist(PPI_nx, f"{PPI_DIRECTORY}/BioGRID_sc.txt", data=False)

# SGD annotations (S. cerevisiae)

In [7]:
SGD_FILENAME = "sgd.gaf.gz"
SGD_FILEPATH = f"{RAW_DATA_DIRECTORY}/{SGD_FILENAME}"

# load SGD gaf-file as dataframe 
with gzip.open(SGD_FILEPATH, 'rt') as gz:
    SGD_df = pd.DataFrame(annotation for annotation in GOA.gafiterator(gz))
    
# Define column with systematic gene names. In SGD this is always the first synonym.
SGD_df['Systematic_ID']  = SGD_df.Synonym.apply(lambda list_:list_[0])

In [21]:
# Filter for proteins
lc_protein_gaf_df = SGD_df[SGD_df['Systematic_ID'].str.startswith('Y')]

# Filter through evidence code
protein_gaf_df = lc_protein_gaf_df[lc_protein_gaf_df['Evidence'].isin(['EXP', 'IDA', 'IPI', 'IMP', 'IGI', 'IEP'])]

# Split into the three GOs
protein_BP_gaf_df = protein_gaf_df[protein_gaf_df['Aspect']=='P']
protein_MF_gaf_df = protein_gaf_df[protein_gaf_df['Aspect']=='F']
protein_CC_gaf_df = protein_gaf_df[protein_gaf_df['Aspect']=='C']

In [22]:
# Get rid of all unnecesarry columns in the GAFs
high_IC_annotations_df = protein_gaf_df[['Systematic_ID', 'GO_ID']].dropna().drop_duplicates()

# Split into the three GOs
high_IC_BP_annotations_df = protein_BP_gaf_df[['Systematic_ID', 'GO_ID']].dropna().drop_duplicates()
high_IC_MF_annotations_df = protein_MF_gaf_df[['Systematic_ID', 'GO_ID']].dropna().drop_duplicates()
high_IC_CC_annotations_df = protein_CC_gaf_df[['Systematic_ID', 'GO_ID']].dropna().drop_duplicates()

### GO DAG extention (Yeast2GO)

In [23]:
# Load obo and gaf files
GO_FILENAME = "go-basic.obo"
GO_FILEPATH = f"{RAW_DATA_DIRECTORY}/{GO_FILENAME}"

# Create annotations with all upstream terms in dict
go_dag = obo_parser.GODag(GO_FILEPATH)
go2parents = {go_id:{*go_dag[go_id].get_all_parents(), go_id} for go_id in go_dag.keys()}

/Users/markusyoussef/Desktop/git/supplements/data/raw_data/go-basic.obo: fmt(1.2) rel(2019-10-07) 47,285 GO Terms


In [24]:
# Create full list/dataframe of annotations
all_annotations_list = [(gene_id, go_term, go_dag[go_term].level) 
                                for _, (gene_id, go_id) in high_IC_annotations_df.iterrows()
                                    for go_term in go2parents[go_id]]
all_annotations_df = pd.DataFrame(
                            all_annotations_list,  
                            columns = ['Systematic_ID', 'GO_ID', 'Level']).drop_duplicates()

# Split into the three GOs
all_BP_annotations_list = [entry for entry in all_annotations_list 
                               if go_dag[entry[1]].namespace == "biological_process"]
all_MF_annotations_list = [entry for entry in all_annotations_list 
                               if go_dag[entry[1]].namespace == "molecular_function"]
all_CC_annotations_list = [entry for entry in all_annotations_list 
                               if go_dag[entry[1]].namespace == "cellular_component"]

all_BP_annotations_df = pd.DataFrame(
                            all_BP_annotations_list,  
                            columns = ['Systematic_ID', 'GO_ID', 'Level']).drop_duplicates()
all_MF_annotations_df = pd.DataFrame(
                            all_MF_annotations_list,  
                            columns = ['Systematic_ID', 'GO_ID', 'Level']).drop_duplicates()
all_CC_annotations_df = pd.DataFrame(
                            all_CC_annotations_list,  
                            columns = ['Systematic_ID', 'GO_ID', 'Level']).drop_duplicates()

### Summary

In [25]:
print("SGD universe sizes:")
print("============================================")
print("Biological Process : " 
    f"{len(set(map(lambda x:x[0],all_BP_annotations_list)))} genes, "
    f"{len(set(map(lambda x:x[1],all_BP_annotations_list)))} GO-IDs ")
print("Molecular Functions: " 
    f"{len(set(map(lambda x:x[0],all_MF_annotations_list)))} genes, "
    f"{len(set(map(lambda x:x[1],all_MF_annotations_list)))} GO-IDs ")
print("Cellular Components: " 
    f"{len(set(map(lambda x:x[0],all_CC_annotations_list)))} genes, "
    f"{len(set(map(lambda x:x[1],all_CC_annotations_list)))}  GO-IDs ")
print('--------------------------------------------')
print("All annotations    : " 
    f"{len(set(map(lambda x:x[0],all_annotations_list)))} genes, "
    f"{len(set(map(lambda x:x[1],all_annotations_list)))} GO-IDs ")

SGD universe sizes:
Biological Process : 4532 genes, 4662 GO-IDs 
Molecular Functions: 3564 genes, 2105 GO-IDs 
Cellular Components: 4013 genes, 904  GO-IDs 
--------------------------------------------
All annotations    : 4967 genes, 7671 GO-IDs 


### Save

In [26]:
all_annotations_df.to_csv(f"{ANNOTATIONS_DIRECTORY}/SGD_sc.csv", index=False)

all_BP_annotations_df.to_csv(f"{ANNOTATIONS_DIRECTORY}/SGD_BP_sc.csv", index=False)
all_MF_annotations_df.to_csv(f"{ANNOTATIONS_DIRECTORY}/SGD_MF_sc.csv", index=False)
all_CC_annotations_df.to_csv(f"{ANNOTATIONS_DIRECTORY}/SGD_CC_sc.csv", index=False)

# BioGRID $\cap$ SGD

In [27]:
PPI_nx = nx.read_edgelist(f"{PPI_DIRECTORY}/BioGRID_sc.txt")

all_annotations_df = pd.read_csv(f"{ANNOTATIONS_DIRECTORY}/SGD_sc.csv")

all_BP_annotations_df = pd.read_csv(f"{ANNOTATIONS_DIRECTORY}/SGD_BP_sc.csv")
all_MF_annotations_df = pd.read_csv(f"{ANNOTATIONS_DIRECTORY}/SGD_MF_sc.csv")
all_CC_annotations_df = pd.read_csv(f"{ANNOTATIONS_DIRECTORY}/SGD_CC_sc.csv")

In [28]:
PPI_annotations_df = all_annotations_df[all_annotations_df.Systematic_ID.isin(PPI_nx.nodes)]

PPI_BP_annotations_df = all_BP_annotations_df[all_BP_annotations_df.Systematic_ID.isin(PPI_nx.nodes)]
PPI_MF_annotations_df = all_MF_annotations_df[all_MF_annotations_df.Systematic_ID.isin(PPI_nx.nodes)]
PPI_CC_annotations_df = all_CC_annotations_df[all_CC_annotations_df.Systematic_ID.isin(PPI_nx.nodes)]

### Summary

In [29]:
print(r"SGD ∩ BioGRID universe sizes:")
print("============================================")
print("Biological Process : " 
    f"{len(set(PPI_BP_annotations_df.Systematic_ID))} genes, "
    f"{len(set(PPI_BP_annotations_df.Systematic_ID))} GO-IDs ")
print("Molecular Functions: " 
    f"{len(set(PPI_MF_annotations_df.Systematic_ID))} genes, "
    f"{len(set(PPI_MF_annotations_df.Systematic_ID))} GO-IDs ")
print("Cellular Components: " 
    f"{len(set(PPI_CC_annotations_df.Systematic_ID))} genes, "
    f"{len(set(PPI_CC_annotations_df.Systematic_ID))}  GO-IDs ")
print('--------------------------------------------')
print("All annotations    : " 
    f"{len(set(PPI_annotations_df.Systematic_ID))} genes, "
    f"{len(set(PPI_annotations_df.Systematic_ID))} GO-IDs ")

SGD ∩ BioGRID universe sizes:
Biological Process : 4496 genes, 4496 GO-IDs 
Molecular Functions: 3537 genes, 3537 GO-IDs 
Cellular Components: 3993 genes, 3993  GO-IDs 
--------------------------------------------
All annotations    : 4923 genes, 4923 GO-IDs 


### Save

In [30]:
PPI_annotations_df.to_csv(f"{ANNOTATIONS_DIRECTORY}/BioGRID-SGD_sc.csv", index=False)

PPI_BP_annotations_df.to_csv(f"{ANNOTATIONS_DIRECTORY}/BioGRID-SGD_BP_sc.csv", index=False)
PPI_MF_annotations_df.to_csv(f"{ANNOTATIONS_DIRECTORY}/BioGRID-SGD_MF_sc.csv", index=False)
PPI_CC_annotations_df.to_csv(f"{ANNOTATIONS_DIRECTORY}/BioGRID-SGD_CC_sc.csv", index=False)